<a href="https://colab.research.google.com/github/Rafikul10/COVID-19/blob/main/COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
#Importing essential libaries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pickle 

In [60]:
from keras.layers import Input,Dense,Flatten,Conv2D,Lambda
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from glob import glob
from keras.layers import MaxPooling1D
from keras.applications.vgg16 import preprocess_input

In [61]:
#Importing the data
IMAGE_SIZE=[224,224]
train_path="/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/train"
test_path="/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/test"

In [62]:
vgg19=VGG19(input_shape=IMAGE_SIZE +[3],weights="imagenet",include_top=False)

80150528/80134624 [==============================] - 1s 0us/step


In [63]:
for layer in vgg19.layers:
  layer.trainable=False

In [64]:
train_data = glob("/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/train/*")

In [65]:
train_data

['/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/train/Viral Pneumonia',
 '/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/train/Covid',
 '/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/train/Normal']

In [66]:
x=Flatten()(vgg19.output)

In [67]:
prediction=Dense(len(train_data),activation="softmax")(x)

In [68]:
model= Model(inputs=vgg19.input,outputs=prediction)

In [69]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [70]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [71]:
from keras.preprocessing.image import ImageDataGenerator

In [72]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    shear_range=0.2,
    rotation_range=40,
    zoom_range=0.2,

)

In [73]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [74]:
training_set =train_datagen.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/train",
                                                target_size=(224,224),
                                                batch_size=16,
                                                class_mode="categorical")

Found 251 images belonging to 3 classes.


In [75]:
testing_set=test_datagen.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/test",
                                             target_size=(224,224),
                                             batch_size=16,
                                             class_mode="categorical")

Found 66 images belonging to 3 classes.


In [76]:
r= model.fit_generator(training_set,
                       validation_data=testing_set,
                       epochs=50,
                       steps_per_epoch=len(training_set),
                       validation_steps=len(testing_set)
                       )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/50
16/16 [==============================] - 243s 15s/step - loss: 1.7759 - accuracy: 0.4821 - val_loss: 0.7326 - val_accuracy: 0.7424
Epoch 2/50
16/16 [==============================] - 218s 14s/step - loss: 0.4844 - accuracy: 0.8367 - val_loss: 0.3518 - val_accuracy: 0.8485
Epoch 3/50
16/16 [==============================] - 219s 14s/step - loss: 0.3734 - accuracy: 0.8805 - val_loss: 0.2189 - val_accuracy: 0.8939
Epoch 4/50
16/16 [==============================] - 219s 14s/step - loss: 0.2832 - accuracy: 0.8884 - val_loss: 0.2327 - val_accuracy: 0.8939
Epoch 5/50
16/16 [==============================] - 222s 14s/step - loss: 0.2122 - accuracy: 0.9203 - val_loss: 0.2888 - val_accuracy: 0.8788
Epoch 6/50
16/16 [==============================] - 220s 14s/step - loss: 0.1972 - accuracy: 0.9363 - val_loss: 0.1257 - val_accuracy: 0.9394
Epoch 7/50
16/16 [==============================] - 218s 14s/step - loss: 0.2081 - accuracy: 0.9124 - val_loss: 0.1443 - val_accuracy: 0.9697
Epoch 

In [77]:
#save it as a h5 file
from keras.models import load_model
model.save("covid_model.h5")

In [78]:
prediction=model.predict(testing_set)

In [79]:
prediction=np.argmax(prediction,axis=1)

In [80]:
prediction

array([1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 1, 1, 0, 2, 1,
       1, 1, 0, 2, 1, 0, 1, 2, 0, 1, 1, 0, 2, 1, 0, 2, 0, 0, 2, 2, 2, 0,
       0, 0, 0, 0, 2, 2, 0, 2, 1, 1, 1, 0, 2, 2, 1, 0, 0, 1, 2, 0, 2, 0])

In [81]:
from keras.models import load_model
from keras.preprocessing.image import image

In [82]:
model=load_model("covid_model.h5")

In [129]:
img=image.load_img("/content/drive/MyDrive/ColabNotebooks/COVID-19/COVID-19(Img)/Covid19-dataset/test/Viral Pneumonia/0103.jpeg",target_size=(224,224))

In [130]:
x=image.img_to_array(img)

In [131]:
x.shape

(224, 224, 3)

In [132]:
x= x/255

In [133]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape


(1, 224, 224, 3)

In [134]:
model.predict(img_data)

array([[0.9644621 , 0.03041307, 0.00512483]], dtype=float32)

In [135]:
a=np.argmax(model.predict(img_data), axis=1)

In [136]:
a

array([0])

In [119]:
if(a==0):
    print("Normal")
elif(a==2):
  print("COVID")    
else:
    print("Viral pneumonia")

covis


In [ ]:
### Create Model from scratch using CNN
#model=Sequential()
#model.add(Conv2D(filters=20,kernel_size=2,padding="same",activation="relu",input_shape=(224,224,1)))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Conv2D(filters=40,kernel_size=2,padding="same",activation ="relu"))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Conv2D(filters=80,kernel_size=2,padding="same",activation="relu"))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Dense(500,activation="relu"))
#model.add(Dense(2,activation="softmax"))
#model.summary()